In [87]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
import pandas as pd

In [72]:
# 1. 브라우저를 실행하고 한솥 사이트로 이동
driver = wb.Chrome()
driver.get("https://www.hsd.co.kr/menu/menu_list")

In [73]:
# 2. 브라우저의 크기를 최대크기로 변경
driver.maximize_window()

In [85]:
# 3. 상품이름, 상품 가격 수집
# 개발자 도구에서 요소를 검사할 때 반드시 화면과 선택한 태그가 일치하는지 확인해볼 것
# 일치하지 않다면 주변 다른 태그들을 분석해서 원하는 요소를 찾기
title = driver.find_elements(By.CSS_SELECTOR, "div > div.item-text > h4")

In [75]:
price = driver.find_elements(By.CSS_SELECTOR, "div > div.item-text > div")

In [76]:
# 3. 더보기 버튼을 틀릭해서 모든 데이터를 로딩받기!
# 버튼이 없는 경우 오류 발생 -> 코드가 통합할 때 밑에 코드가 동작을 안함
# 이떄 필요한게 예외 처리 (오류가 나기 전까지 코드를 실행하다가, 오류가 나면 멈추지 않고 다른 코드가 동작.
# btn = driver.find_element(By.CSS_SELECTOR, "#btn_more > span > a")
# btn.click()

In [86]:
len(title)

102

In [78]:
# 컴퓨터에게 시간을 부여할 때 사용하는 라이브러리
# -> 너무 빠르게 요청해버리면 서버가 올바르지 않는 응답하기 때문에
import time

In [82]:
try :
    for i in range(10) :
        btn = driver.find_element(By.CSS_SELECTOR, "#btn_more > span > a")
        btn.click()
        #잠깐 잠들어!
        time.sleep(1)
except :
    print("더보기 완료")


더보기 완료


# 예외처리 할 때 주의점
1) 동작에 관련된 예외처리 (더보기 버튼) 에러나면 그냥 출력문을 통해서 넘어가도 상관없음
2) 데이터를 수집할 때는 반드시 에러가 나는 부분에 null, 없음, 0 등과 같은 다른 데이터를 넣어줘야 한다.
   - 데이터의 개수가 다르면 표, DB, 파일로 제작할 때 오류가 발생 -> 데이터의 개수가 다르기 때문에    

In [88]:
# 코드 통합
# 코드의 중간중간 time.sleep()이 필요하다 -> 로딩 받는 시간이 필요해서
# 클라이언트가 서버에게 요청을 할 때마다 시간을 부여한다.
driver = wb.Chrome()
driver.get("https://www.hsd.co.kr/menu/menu_list")
time.sleep(1)
driver.maximize_window()
try :
    for i in range(10) :
        btn = driver.find_element(By.CSS_SELECTOR, "#btn_more > span > a")
        btn.click()
        #잠깐 잠들어!
        time.sleep(1)
except :
    print("더보기 완료")
title = driver.find_elements(By.CSS_SELECTOR, "div > div.item-text > h4")
price = driver.find_elements(By.CSS_SELECTOR, "div > div.item-text > div")
titleList = []
priceList = []
for i in range(len(title)):
    titleList.append(title[i].text)
    priceList.append(price[i].text)
dic = {"상품이름":titleList, "가격정보":priceList}
data = pd.DataFrame(dic)
data.to_csv("한솥도시락.csv", encoding = "euc-kr")

더보기 완료


# 이번 예제에서 학습 내용
1) 반드시 데이터를 수집하기 전에 데이터 로딩이 먼저 일어나야 한다. (더보기, 스크롤 내리기)
2) 코드를 통합할 때는 반드시 중간중간 time을 부여해야 한다. -> 클라이언트와 서버가 데이터를 주고 받을 때마다 넣어줘야 한다.
3) 페이지마다 구조가 다 다르기 때문에, 반드시 예외처리를 해야한다.
   - 동작을 할 때는 (클릭, 엔터 등) 예외처리 문법에 그냥 print, pass 사용
   - 데이터를 수집할 때는 데이터가 없는 경우, 반드시 null, 0 등과 같은 다른 데이터를 넣어줘야 한다, -> 개수 맞추기 위해서이다. => 올바르게 표 , DB, 파일로 저장할 때 오류 발생을 막기 위해서!
4) 크롤링의 프로세스
   1) 창을 실행한다, -> 특정 사이트로 이동
   2) 데이터 로딩을 진행한다. -> 더보기나 스크롤 등...
   3) 요소 수집 -> 텍스트로 변환
   4) 데이터 활용 -> 표, 파일 제작